In [1]:
pip install z3-solver

     |████████████████████████████████| 30.5MB 139kB/s 


In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random
from z3 import *

In [3]:
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense
 
#sklearn.model_selection.train_test_split(*arrays, **options)
from sklearn.model_selection import train_test_split
 
from PIL import Image, ImageOps
from keras.layers.convolutional import Conv2D
from matplotlib import cm


In [4]:
from google.colab import drive
drive.mount('/content/drive')
fold = "drive/My Drive/Colab Notebooks/smtdatatrials2/trials2/" 
import sys
sys.path.insert(1, fold)


Mounted at /content/drive


In [5]:
from tensorflow.keras.models import load_model

Loading Up a Model from smt_project_nn

In [6]:
model = load_model(fold+'models')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               22500     
_________________________________________________________________
dense_2 (Dense)              (None, 40)                4000      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                400       
_________________________________________________________________
dense (Dense)                (None, 4)                 40        
Total params: 26,940
Trainable params: 26,940
Non-trainable params: 0
_________________________________________________________________


In [8]:
weight1=[]
#biase1=[]
for layer in model.layers:
    weight = layer.get_weights()[0]
    #bias = layer.get_weights()[1] 
    weight1.append(weight)
    #biase1.append(bias)


wbpairs = model.get_weights()
#print(wbpairs[0])
print('Weight1[0]=',weight1[0],'\n\n',len(weight1[0]))
#print('biase1[0]=',biase1[0])

Weight1[0]= [[ 3.6127035e-02  5.6199886e-02  6.6610478e-02 ...  4.4020712e-02
   9.6576318e-02  4.5049138e-02]
 [-9.1441702e-03  6.9404088e-02 -5.7055060e-02 ... -3.5717983e-02
   7.3694386e-02 -8.9444183e-02]
 [ 1.3132227e-02 -4.6205945e-02  5.9141043e-05 ... -1.1069778e-01
  -5.8497082e-02  7.6235548e-02]
 ...
 [ 7.4387319e-02 -2.3392579e-02  1.1829632e-01 ...  7.7394904e-03
   1.2969336e-01 -9.1654070e-02]
 [-7.9071850e-02  4.4572237e-03 -1.6070472e-02 ... -7.7395856e-02
   1.2861602e-01 -9.2867702e-02]
 [ 4.5015711e-02  1.0438054e-02  7.8057490e-02 ... -2.5746576e-02
  -3.4494568e-02  5.3635169e-02]] 

 225


In [9]:
#Pre-merging. Convert so that only one output node
#note:currently has softmax in output layer. change for actual abstraction



# Say, the last layer has m neurons. The network classifies as the kth class if the kth neuron's value 
#is greater than all the others.

def isclasskmaker(n,k): #n=number of classes
  #make another layer of size n, Relu. The ith neuron of this layer will have input weights [l0,l2....,ln-1] where lk=-1, li=1.. and everything else = 0
  #This means that the result is max(0,li-lk). This means that if the kth neuron's weight is greater than every other weight
  #then every neuron in our new layer will be zero. Otherwise, at least one neuron will have a value greater than 1.
  
  # Then, for the new final layer, have just 1 neuron, and the incoming weights being 1. The value =0 means that the network predicted k-th class, else 
  #predicted something else.
  nweights = []
  if k>=n:
    return('invalid class')
  if n==1:
    return("Only one output node")
  default = []
  for i in range(n):
    default.append(0)
  default[k] = -1
  for i in range(n):
    wti = []+default
    if i!=k:
      wti[i] = 1
    else:
      wti[i] = 0
#    print('default:',default)
    a = np.array(wti)
    nweights.append(a)

  nww1 = np.array(nweights)
  nww = np.transpose(nww1)
  for i in range(n):
    p = np.array([1])
    default[i]= p

  d1 = np.array(default)

  return(nww,d1)


  

isclasskmaker(4,0)

#So, x,y = isclasskmaker(n,k). Append x,y to the wieghts, array-ify to get a network which outputs 0 iff the original predicted kth class and >0 else.





(array([[ 0, -1, -1, -1],
        [ 0,  1,  0,  0],
        [ 0,  0,  1,  0],
        [ 0,  0,  0,  1]]), array([[1],
        [1],
        [1],
        [1]]))

#Abstraction

In [10]:
#toy example(in slides)

def id(wprev,wnext,pns,ids):            #-wprev-->  (.) -wnext--->
  nl = len(ids)#=len(pns)=len(wl[0])  
  tlayer = len(wnext)
  idnew=[]
  pnnew=[]
  wprevt = np.transpose(wprev)                                      
  toweightsupdate=[]
  newnodes = [] #and weights from these
###########    
  corresponding = []
###########
  for i in range(tlayer):
    weights_from_node = wnext[i]
    weights_to_node = wprevt[i]
    pnode = []
    nnode = []
    counterp=0
    countern=0
#    print('whts_from_node=wnext[i]=',weights_from_node)
    for j in weights_from_node:           #for j in weights from node
      ###############
#      corr=[]
      ###############
      if j>=0:
        pnode.append(j)
        nnode.append(0)
        counterp=counterp+1
      else:
        nnode.append(j)
        pnode.append(0) ##########################
#      creates + and - variant of node
        countern=countern+1
#      print('i,j=',i,'_',j,pnode,nnode)
    if counterp == len(weights_from_node): #means all of nnode is 0
      newnodes.append(pnode)
      toweightsupdate.append(weights_to_node)
      pnnew.append(1)
      ################
      corresponding.append(i)
      ################
    elif countern == len(weights_from_node):
      newnodes.append(nnode)
      toweightsupdate.append(weights_to_node)
      pnnew.append(-1)
      ################
      corresponding.append(i)
      ################
    else:
      newnodes.append(pnode)                  
      newnodes.append(nnode)
      pnnew.append(1)
      pnnew.append(-1)
      ################
      corresponding.append(i)
      corresponding.append(i)
      ################
      toweightsupdate.append(weights_to_node)
      toweightsupdate.append(weights_to_node) #need to transpose
#    corresponding.append(corr)

  #------ID imp
  #newnodes, pnnew,toweightsupdate.

#  idnew
  new_newnodes = []
#  print('newnodes',newnodes)
#  print('pnnew',pnnew)
  toweights2=[]
  pnnew2=[]

#  print(corresponding,newnodes)
  corresponding2=[]
  for i in range(len(newnodes)):
    thiscor = corresponding[i]
    inode=[]
    dnode=[]
    weights_to_node = toweightsupdate[i]
    icounter = 0
    dcounter = 0
    for j in range(len(newnodes[i])):
#      print('pnnew',pnnew)
#      print('\nids',ids)
#      print('i,j:',i,',',j)
#      print('checkinasf;afafeadfslsfkblj;dsfjb.kas',pnnew[i],ids[j])
      if (((pnnew[i]==1) & (ids[j]==1)) | ((pnnew[i]==-1) & (ids[j]==-1))):
        inode.append(newnodes[i][j])
        dnode.append(0)
        icounter=icounter+1
#        print('herefirst')
      elif (((pnnew[i]==1) & (ids[j]==-1)) | ((pnnew[i]==-1) & (ids[j]==1))):
        dnode.append(newnodes[i][j])            
        inode.append(0)
        dcounter = dcounter+1
#       print('here')
#    print('dfsgdfssdfdds',inode,dnode,'-----------------',newnodes[i])
    aaa = 0
    for t in inode:
      if t == '_':
        aaa = aaa+1
    bbb = 0
    for t in dnode:
      if t == '_':     #!
        bbb = bbb+1
    
    if icounter>0:
      if aaa!=len(inode):
        pnnew2.append(pnnew[i])
    if dcounter>0:
      if bbb!=len(dnode):
        pnnew2.append(pnnew[i])
#   print(icounter,dcounter,len(newnodes[i]))
    if icounter==len(newnodes[i]):
      if aaa!=len(inode):
        new_newnodes.append(inode)
        toweights2.append(weights_to_node)
        idnew.append(1)
#####################
        corresponding2.append(thiscor)
#####################
    elif dcounter==len(newnodes[i]):
      if bbb!=len(dnode):
        new_newnodes.append(dnode)
        toweights2.append(weights_to_node)
        idnew.append(-1)
#####################
        corresponding2.append(thiscor)
#####################
    else:
      if aaa!=len(inode):
        new_newnodes.append(inode)
        idnew.append(1)
        toweights2.append(weights_to_node)
#####################
        corresponding2.append(thiscor)
#####################
      if bbb!=len(dnode):
        idnew.append(-1)
        new_newnodes.append(dnode)
        toweights2.append(weights_to_node)
#####################
        corresponding2.append(thiscor)
#####################      
#  print('id_iteration_complete')
#  print(corresponding2,new_newnodes)
  return np.transpose(toweights2),new_newnodes,pnnew2,idnew,corresponding2 #remember totranspose toweights 2 before actually using this .....(done)


#(weights into layer,weights out of layer, +/- of next layer,I/D of next layer ) --> (wts into updtd layer,weights out of updtd layer,+/- status of updated layer,I/D of u.l)
id([[1,-1,0]],[[1,-1],[0,12],[-1,-1]],[1,1],[1,-1])






#also outputs corresponding 2... which gives which neurons from the original layer the new one corresponds to.



(array([[ 1,  1,  1,  1, -1, -1,  0,  0]]),
 [[1, 0], [0, 0], [0, -1], [0, 0], [0, 0], [0, 12], [0, -1], [-1, 0]],
 [1, 1, -1, -1, 1, 1, -1, -1],
 [1, -1, 1, -1, 1, -1, 1, -1],
 [0, 0, 0, 0, 1, 1, 2, 2])

The above can be used to create the split network by recursively applying on the layers of the original.

In [11]:
def splitnetwork(weights):
#[...(array([[ 0, -1],[ 0,  1]]), array([[1],[1]]))].
  newnetwork = []
  ids=[]
  pns=[]
  corresponding_lv=[]
  if len(weights)==0:
    print('no weights given')
    return None
  idfl = []
  pnfl = []
  tt = len(weights[-1][0])#number of final neurons


  for i in range(tt):
    idfl.append(1)
    pnfl.append(1)
  ids.append(idfl)
  pns.append(pnfl)  
  if len(weights)==1:
    print('no hidden layer')
    return weights,pns,ids # we actually want to return the reverse ids and pns since we're going backwards, but length is 1 so it's fine

#id: (weights into layer,weights out of layer, +/- of next layer,I/D of next layer ) --> (wts into updtd layer,weights out of updtd layer,+/- status of updated layer,I/D of u.l)
  #len(weights)>=2
  final_layer_corr=[]
  for j in range(len(weights[-1][0])):
    final_layer_corr.append(j)
  corresponding_lv.append(final_layer_corr)
#  print('dsafdfsaf',corresponding_lv)
  weights_into_layer = weights[-2]
  weights_out_of_layer = weights[-1]
  pn_next_layer = pns[-1]
  id_next_layer = ids[-1] #final layer.
  for i in range(1,len(weights)): # travel from -1 ---> -(len(weights))
    weights_into_layer = weights[-i-1]
#    print(weights_into_layer,weights_out_of_layer,pn_next_layer,id_next_layer,'---id------------------->',id(weights_into_layer,weights_out_of_layer,pn_next_layer,id_next_layer))
    weights_to,weights_from,pn_this,id_this,corresponding2 = id(weights_into_layer,weights_out_of_layer,pn_next_layer,id_next_layer)
    ids.append(id_this)
    pns.append(pn_this)
    newnetwork.append(weights_from)
#    print('weights_from',id_this)
    corresponding_lv.append(corresponding2)

    weights_out_of_layer = weights_to
    pn_next_layer = pn_this
    id_next_layer = id_this
#    print('-i',-i,'weights[-i-1]',weights[-i-1])
#    reversing.

  newnetwork.append(weights_out_of_layer)
  newnetwork.reverse()
  corresponding_lv.reverse()
  pns.reverse()
  ids.reverse()
#  print(pns)
#  print(ids)
#  print(corresponding_lv)
  triple=[]
  for i in range(len(ids)):
    tl=[]
    thislayer=ids[i]
    for j in range(len(thislayer)):
      tl.append([(pns[i][j],ids[i][j],corresponding_lv[i][j])])
    triple.append(tl)

#  return newnetwork,pns,ids,corresponding_lv,record,triple  #need to reverse
  return newnetwork,triple


#mock network

network = [[[1]], [[2, 1]], [[1], [-1]]]
print('ognetwork:',network)
x,y=splitnetwork(network)
print('splitnetwork:',x,'its +/-,I/D,corresponding node values:',y)

ognetwork: [[[1]], [[2, 1]], [[1], [-1]]]
splitnetwork: [array([[1, 1]]), [[2, 0], [0, 1]], [[1], [-1]]] its +/-,I/D,corresponding node values: [[[(1, 1, 0)], [(1, -1, 0)]], [[(1, 1, 0)], [(-1, -1, 1)]], [[(1, 1, 0)]]]


When merging, we have to take sums,max,min. But we have '_'. These have to avoided. 

In [12]:


def elementwise_sum(L):#[[x1,x2],[y1,y2]]->[[x1+y1,x2+y2]]
  ret =[]
  if len(L)==0:
    print('empty list')
    return None
  else:
    for i in range(len(L[0])):
      sum=0
      for j in range(len(L)):
        if L[j][i] !='_':
          flfy = float(L[j][i])
          sum=sum+flfy
      ret.append(sum)
    return ret 

print(elementwise_sum([[]]))


def elementwise_max(L):#[[x1,x2],[y1,y2]]->[max(x1,y1),max(x2,y2)]
  if len(L)==0:
    print('empty list')
    return None
  else:
    ret=[]
#  print('L=',L)
    for i in range(len(L[0])):
      maxn=np.NINF
      for j in range(len(L)):
        if L[j][i]!='_':
#        print('inmyelmmax:',L[j][i],'type', type(L[j][i]))
          flfy = float(L[j][i])
          if flfy >maxn:
            maxn= flfy
      ret.append(maxn)
    for i in range(len(ret)):
      if ret[i]==np.NINF:
        ret[i]=0
    return ret  #note... can return -inf if the ith element of each list is '_', but this can't happen in NN because at least one incoming weight has to be nonempty

#print(elementwise_max([['_','_'],[1,'_'],[-12,23]]))

def elementwise_min(L):#[[x1,x2],[y1,y2]]->[min(x1,y1),min(x2,y2)]
  if len(L)==0:
    print('empty list')
    return None
  else:
    ret=[]
#  print('L',L)
    for i in range(len(L[0])):
      minn=np.inf
      for j in range(len(L)):
#      print("Lji",L[j][i])
        if L[j][i]!='_':
          flfy = float(L[j][i])
#        print('flfy',flfy,type(flfy))
          if flfy<minn:
            minn=flfy
#      print(minn)
      ret.append(minn)
    for i in range(len(ret)):
      if ret[i]==np.inf:
        ret[i]=0
    return ret
print(elementwise_sum([['1','1'],[1,'0'],[-12,23]]))

[]
[-10.0, 24.0]


Merging

...Below, doing merging for just one layer(#). Partition-M

In [13]:
def partition_merge(weights_to_layerut,weights_from_layer,info3,P):
  weights_to_layer = np.transpose(weights_to_layerut)
#  print(weights_to_layer)
  dict_info={}
#  print('info3:',info3)
  for i in range(len(info3)):                 #info3 is likea default partition of a layer. It basically just gives you the labelling. so [ [(1,1,0)] , [(1,-1,1)] ]------(1)
    t = info3[i]                              #means the 0th neuron in this layer is +,I and the 1st neuron in this lauer is +,D.
#    print('t=',t)                            #t takes such a labelling, and dict_info[t[0]] assigns the incomming, and outgoing weights of said neuron.
    dict_info[t[0]]=(weights_to_layer[i],weights_from_layer[i]) #the incoming and outgoing edges of each neuron (i,j,k)

  #assume consistency in P(should be there if done correctly)

  #updated_info=[]
  updated_from=[]
  updated_to=[]
  #print('partition_merge:p=',P)


  for i in P:                   #P is the partition we're mergin this to. So, in the above(1), if P=[[(1,1,0),(1,-1,1)]] we're merging the neurons (1,1,0) and(1,-1,1) into one.

    for j in i:                 #i a 'new neuron', and j is iterating over the neurons of the original contained in(merged into) this new neuron
      set_pn,set_id=j[0],j[1]   #Assuming that each merging is consistent, we just need to check if the 1st neuron to be merged into a partitio is +/I or whatever. 
      break                     #set_pn stores the +/- value, and set_id stores the I/D value.
    
    new_to=[]                   #have to update the weights to this new/merged node using set_pn,set_id values
    new_from=[]                 #...
    #new_cor=[]

    for j in i:
      #new_cor.append(j)
      weights_to_node,weights_from_node = dict_info[j]#weights_to_node gives a list of weights that go into the node j, likewise weights_from_node
      new_to.append(weights_to_node)                  #list of lists containing weights_to_node where the nodes are ones to be merged
      new_from.append(weights_from_node)              #...
    #print(i,'new_to:',new_to,'new_from:',new_from)
    if set_id==1:
      updated_to.append(elementwise_max(new_to)) #returns list of weights to the merged node

    else:
      updated_to.append(elementwise_min(new_to))#returns list of weights to the merged node
    
    updated_from.append(elementwise_sum(new_from))
    #updated_info.append(new_cor)
    #updated_info.append(i)

  updated_to1=np.transpose(updated_to)
  return updated_to1,updated_from,P #gives weights to this merged layer from the previous, the weights from this layer, as well as the nodes that were used
                                    #to merge and get each neuron in this merged layer.





#input: (weights_to_layerut,weights_from_layer,info3,P) info gives (+/-,I/D,corresponding node in og)
partition_merge([[1,4,2],[-2,-1,-3]],[[5],[3],[4]],[[(1,1,0)],[(1,1,1)],[(1,1,2)]],[[(1,1,0),(1,1,1),(1,1,2)]])

(array([[ 4.],
        [-1.]]), [[12.0]], [[(1, 1, 0), (1, 1, 1), (1, 1, 2)]])

Merging network

eg: ognetwork: [[[1]], [[2, 1]], [[1], [-1]]]
splitnetwork: [array([[1, 1]]), [[2, 0], [0, 1]], [[1], [-1]]] its +/-,I/D,corresponding node values: [[[(1, 1, 0)], [(1, -1, 0)]], [[(1, 1, 0)], [(-1, -1, 1)]], [[(1, 1, 0)]]]

In [14]:
def partition_abstraction(weights,infox,partition):#note: partition will include all non-inital nodes as we're only labelling those
  if len(weights)<=1:                              #likewise for infox
    print('No hidden layer')
    return weights
  
  abstracted_weights=[]
  weights_to2 = weights[-1]
  #weights_to_layer = weights[-2]
#  rpart=[]
#  rpart.append(partition[-1])

  for i in range(1,len(weights)): 
    weights_from_layer = weights_to2
    weights_to_layer = weights[-i-1]
    p = partition[-i-1] #note: partition -1 is the final layer, and 0 is the 1st hidden-layer
    info3 = infox[-i-1]


    weights_to2,weights_from2,info = partition_merge(weights_to_layer,weights_from_layer,info3,p)
    #note by how partition_merge is defined, info=p...  

    abstracted_weights.append(weights_from2)
#    rpart.append(info)
#    print('merged..loop:',i)

  abstracted_weights.append(weights_to2)
  abstracted_weights.reverse()
#  rpart.reverse()
  arified=[]
  for i in abstracted_weights:
    arified.append(np.array(i))
  
#  return arified,rpart                                         #have commented all rpart things..don't need it..maybe
  return arified,partition#rpat=partition?
  
#mock network                                                                   
splitn =   [[[1, 1]], [[2, 0], [0, 1]], [[1], [-1]]]                                           
inx = [[[(1, 1, 0)], [(1, -1, 0)]], [[(1, 1, 0)], [(-1, -1, 1)]], [[(1, 1, 0)]]]
ptn = [[[(1, 1, 0)], [(1, -1, 0)]], [[(1, 1, 0)], [(-1, -1, 1)]], [[(1, 1, 0)]]]                        
partition_abstraction(splitn,inx,ptn)#work with splitted version

([array([[1., 1.]]), array([[2., 0.],
         [0., 1.]]), array([[ 1.],
         [-1.]])],
 [[[(1, 1, 0)], [(1, -1, 0)]], [[(1, 1, 0)], [(-1, -1, 1)]], [[(1, 1, 0)]]])

In [15]:
def abstraction(weights,infox):
  #basically takes the split network(given by weighs), and partition_abstraction is done wrt the partition given by the 
  # paper 
  #nnet,triple = splitnetwork(weights)   #split network is equivalent to the original
  partition = []


#--------------edit
#initially was just one input(splitting was done within this function).. but now we assume the input is the split network, along with the corresponding with the initial
#partition. So. triple=infox
#------------------

  for i in infox: # i is a layer.. so a list of list like [ [(1,1,0)] ,...]
    pi=[]
    pd=[]
    ni=[]
    nd=[]
    t=[]
    for j1 in i: #j1 is a neuron... in this default case only contains one element each [(1,1,0)]
      j=j1[0] # j is like (1,1,0)
      if j[0]==j[1]:
        if j[0]==1: #if +,I
          pi.append(j)
        else:       #if -,D
          nd.append(j)
      else:
        if j[0]==1: #if +,D
          pd.append(j)
        else:       #if -,I
          ni.append(j)
    if len(pi)!=0:
      t.append(pi)
    if len(pd)!=0:
      t.append(pd)
    if len(ni)!=0:
      t.append(ni)
    if len(nd)!=0:
      t.append(nd)
    partition.append(t)                 #have created the partition as corresponding to the thing in the paper
#  print('partition:',partition)
#  print('triple:',triple)  
  return  partition_abstraction(weights,infox,partition)



splitn = [[[1, 1]], [[2, 0], [0, 1]], [[1], [-1]]]
inx = [[[(1, 1, 0)], [(1, -1, 0)]], [[(1, 1, 0)], [(-1, -1, 1)]], [[(1, 1, 0)]]]
print('splitognetwork:',splitn)
a1,a2=abstraction(splitn,inx)
print('abstraction network:',a1, 'new partition?:',a2)



#use this funciton for the first abstract network creation. Then, in the refine algo, use partition_abstraction, triple is the value you get using splitnetwork.
#might as well store it.

#partition has to be generated

splitognetwork: [[[1, 1]], [[2, 0], [0, 1]], [[1], [-1]]]
abstraction network: [array([[1., 1.]]), array([[2., 0.],
       [0., 1.]]), array([[ 1.],
       [-1.]])] new partition?: [[[(1, 1, 0)], [(1, -1, 0)]], [[(1, 1, 0)], [(-1, -1, 1)]], [[(1, 1, 0)]]]


Have done abstraction.

In [89]:
def refine(ognetwork,infox,abnetwork,abpart,cex):     #again.. the ognetwork is assumed to be split
                                                      #(used the 'triple' instead of infox...)
  #cex is a list of list of 225 values.[[0],[0]...]
  og_val=[]
  ab_val=[]
  og_val.append(cex)
  ab_val.append(cex)
  run=len(ognetwork)

  for i in range(run):
    ogi=og_val[-1]                                    #ogi is supposed to store the values ognetwork has in each neuron after applying the ith set of weights 
    abi=ab_val[-1]                                    #...

    og_weights=np.transpose(ognetwork[i])             #og_weight[j] gives the weights going into the jth neuron
    ab_weights=np.transpose(abnetwork[i])
    
    ognewval=[]                               
    abnewval=[]                                       #these are supposed to give the values in the next layer.  
#    print('i:',i)
#    print('ogi:',ogi)
#    print('og_weights:',og_weights)                    
    for j in range(len(og_weights)):
                                                      #j is used to represent a single neuron in the next layer
      t1 = 0
      for l in range(len(ogi)):                       #len(ogi) is the number of inputs per neuron..... i.e number of neurons in the current layer
        t1=t1+((og_weights[j][l])*(ogi[l][0]))
      t2=max(t1,0)
      ognewval.append([t2])
    og_val.append(ognewval)

    for j in range(len(ab_weights)):
      t1=0
      for l in range(len(abi)):
        t1=t1+((ab_weights[j][l])*(abi[l][0]))
      t2=max(t1,0)
      abnewval.append([t2])
    ab_val.append(abnewval)

#  print('ogval:',og_val)
#  print('abval:',ab_val)

  #og_val=[[values],[],[]..]..og_val[i] for the ith layer


#  print('infox(inrefine):',infox)
#  print('length of infox:',len(infox))
#  print('ogval:',og_val)
  og_dict={}
  for i in range(len(infox)):
    og_dict[i+1]={}                                       #the i+1th layer(starting from 1) will have a dictionary that points to each node which in turns knows the 
                                                          #values of said node
    for j in range(len(infox[i])):
#      print('tripleij:',triple[i][j])
      og_dict[i+1][infox[i][j][0]]=og_val[i+1][j][0]      #ogval stores values of each node.. but starts at the 0th(inital) layer, but infox only starts from the 1st layer
                                                        	#so...og_val[i+1] is used
  
                                                          #og_dict[1][(1,1,0)] gives the value in the neuron (1,1,0) in the 1st layer.


  bad_l=[]
#  print('og_dict=',og_dict,'\n\n')
  for i in range(len(abpart)-1):                          #don't consider the final node. So abpart[i] can be any hidden layer's partition. like:[ [..(1,1,0),(1,1,1)...] , [....] ]
#    print('abi:',abpart[i])
    big_diff=0
#    print('biggg_diff_inloopstat:',big_diff)
    bad_nl=(0,0,0)#########
    
    for j in range(len(abpart[i])):
      len_part = len(abpart[i][j])                        #we only want to check diff in parts where more than one neuron exists               
      abnode_val = ab_val[i+1][j][0]

#      print('abij:',abpart[i][j])
      for k in abpart[i][j]:                            #abpart[i][j] is a list of triples, giving the corresponding nodes in the original network
                                                        #k gives one such node(represented by a triple):

#        print(i+1,'..',k,'jhygtfgcvbnmjv fgrrgvrvr vrr tbttr    tvtrvgtrvgrv',og_dict[i+1][k])
    
        corresponding_ognode_val=og_dict[i+1][k]        #abpart[0] corresponds to the 1st layer, since we don't have a partition for the initial states,.. likewise
                                                        #so... need og_dict[i+1]
        diff = abs(abnode_val-corresponding_ognode_val)
        if (diff>big_diff) and (len_part>1):                               #note that bad_nl=(0,0,0)-> this is not a real node, but bad_nl will be replaced by a real node iff there is a non-zero
          big_diff=diff                                 #difference.
          bad_nl=k
        else:
          pass                                          #for an i, big_diff after this loop gives the largest difference
                                                        #and bad_nl gives the corresponding node for that...
#    print('big_diff:',big_diff,'\ncorrrrrrr:',bad_nl)
    bad_l.append([(big_diff,bad_nl)])                   #so, bad_l[i] says layer i+1(since we're not considering the initial(or final)) layer has the largest difference
                                                        # equal to big_diff due to merging the node bad_nl of the original(split) network.


  print('\nbad_l:',bad_l)
                                                        #bad_l contains a list of list of (diff,(node)) pairs.


#  nch=0
  max_diff_in_layer=0
  for i in range(len(bad_l)):
    t=bad_l[i][0][0]
    if t>max_diff_in_layer:                           #store which node has the largest difference, as well as the layer its in
      c_node_in_layer =bad_l[i][0][1]
      max_diff_in_layer=bad_l[i][0][0]
      m_layer = i                                   
#    nch=nch+max_diff_in_layer
#  print('badl:',bad_l)
#  print('m_layer:',m_layer)
#  print('nch:',nch)  
#  print('c_node_in_layer:',c_node_in_layer)
  if max_diff_in_layer==0:
    print('No refinement possible')
    llist=[]
    for i in abpart:
      llist.append(len(i))
    print(llist)
    return abnetwork,abpart
  else:
    #change abpart[i=m_layer]; split c_node_in_layer
#    print('abpart_before_split:',abpart)

    foundit=0
    for i in abpart[m_layer]:
#      print('dfghjklsdfghbjkmlzsxdfcgvhbjkmazsxdfcgvbhjkmawsdfghjkmswedfghjkml,',i)
      for j in i:
        if j == c_node_in_layer:
          print('m_layer',m_layer,'---inrefine->tbs j:',j)                                                #find the node with the biggest difference,
          i.remove(j)                                                           #remove it
          toappend=j
          foundit=1
          break
        else:
          pass
      if foundit==1:
        break
    abpart[m_layer].append([toappend])                                          #append it back, but keep it by itself
#    print('abpart_after_split:',abpart)
#  print('1:=========',ognetwork,'\n2:...................',infox,'\n3+++++++++++++++',abpart)
  return partition_abstraction(ognetwork,infox,abpart)








ogn = [[[1, 1]], [[2, 0], [0, 1]], [[1], [-1]]]
ogp = [[[(1, 1, 0)], [(1, -1, 0)]], [[(1, 1, 0)], [(-1, -1, 1)]], [[(1, 1, 0)]]]
agn,agp=abstraction(ogn,ogp)
#print('\n ogn:',ogn,'\n ogp:',ogp,'\n agn:',agn,'\n agp:',agp)
#print('og_network_split:\n',ogn)
print('\nabstracted_initial:\n',agn)
ref,rpa =refine(ogn,ogp,agn,agp,[[3]])
print('\nrefined, with respect to [[3]]:\n', ref,'.!..!..',rpa)


abstracted_initial:
 [array([[1., 1.]]), array([[2., 0.],
       [0., 1.]]), array([[ 1.],
       [-1.]])]

bad_l: [[(0, (0, 0, 0))], [(0, (0, 0, 0))]]
No refinement possible
[2, 2, 1]

refined, with respect to [[3]]:
 [array([[1., 1.]]), array([[2., 0.],
       [0., 1.]]), array([[ 1.],
       [-1.]])] .!..!.. [[[(1, 1, 0)], [(1, -1, 0)]], [[(1, 1, 0)], [(-1, -1, 1)]], [[(1, 1, 0)]]]


#Model...

In [90]:
#len(model.get_weights()[0])
weight_mod = model.get_weights()
is0 = isclasskmaker(4,0)
for i in is0:
  weight_mod.append(i)
print('model with 1 output:',weight_mod)
split_wm,split_infox = splitnetwork(weight_mod)

model with 1 output: [array([[ 3.6127035e-02,  5.6199886e-02,  6.6610478e-02, ...,
         4.4020712e-02,  9.6576318e-02,  4.5049138e-02],
       [-9.1441702e-03,  6.9404088e-02, -5.7055060e-02, ...,
        -3.5717983e-02,  7.3694386e-02, -8.9444183e-02],
       [ 1.3132227e-02, -4.6205945e-02,  5.9141043e-05, ...,
        -1.1069778e-01, -5.8497082e-02,  7.6235548e-02],
       ...,
       [ 7.4387319e-02, -2.3392579e-02,  1.1829632e-01, ...,
         7.7394904e-03,  1.2969336e-01, -9.1654070e-02],
       [-7.9071850e-02,  4.4572237e-03, -1.6070472e-02, ...,
        -7.7395856e-02,  1.2861602e-01, -9.2867702e-02],
       [ 4.5015711e-02,  1.0438054e-02,  7.8057490e-02, ...,
        -2.5746576e-02, -3.4494568e-02,  5.3635169e-02]], dtype=float32), array([[ 0.19373448, -0.18705568, -0.10012381, ..., -0.17807184,
         0.1522985 , -0.00381974],
       [-0.17776063,  0.17617904, -0.1218728 , ..., -0.16635543,
         0.13369158, -0.02622381],
       [ 0.16712669, -0.09214905, -0.1404

In [38]:
abstracted_model_weights,abstracted_model_dp = abstraction(split_wm,split_infox)
print(abstracted_model_weights)
#for i in abstracted_model_weights:
#  arified.append(np.array(i))
#print(arified)
#print(len(arified))
#abstracted_model_weights = arified

[array([[ 0.14215901, -0.13681966,  0.14215901, -0.13681966],
       [ 0.14401719, -0.12869945,  0.14401719, -0.12869945],
       [ 0.14765017, -0.13447265,  0.14765017, -0.13447265],
       [ 0.14445633, -0.13118415,  0.14445633, -0.13118415],
       [ 0.14312282, -0.13909   ,  0.14312282, -0.13909   ],
       [ 0.13731728, -0.1401217 ,  0.13731728, -0.1401217 ],
       [ 0.14017038, -0.14155152,  0.14017038, -0.14155152],
       [ 0.13763078, -0.13802716,  0.13763078, -0.13802716],
       [ 0.14122036, -0.13192227,  0.14122036, -0.13192227],
       [ 0.13262436, -0.13252185,  0.13262436, -0.13252185],
       [ 0.13905184, -0.15310161,  0.13905184, -0.15310161],
       [ 0.13556994, -0.13169284,  0.13556994, -0.13169284],
       [ 0.13712987, -0.14571905,  0.13712987, -0.14571905],
       [ 0.15216616, -0.14105581,  0.15216616, -0.14105581],
       [ 0.13766539, -0.13996999,  0.13766539, -0.13996999],
       [ 0.1457385 , -0.14022683,  0.1457385 , -0.14022683],
       [ 0.14451531, -0

In [91]:
#assumption.. at least 1 hidden layer
ab_model = Sequential()
inlen = len(abstracted_model_weights[0])
l1 = len(abstracted_model_weights[1])
#print(inlen,l1)
ab_model.add(Dense(l1, input_dim=inlen, activation='relu',use_bias=False))
for i in range(2,len(abstracted_model_weights)):
#  print(len(abstracted_model_weights[i]))
  ab_model.add(Dense(len(abstracted_model_weights[i]),activation='relu',use_bias=False))
lf= len(abstracted_model_weights[-1][0])
ab_model.add(Dense(lf,activation='relu',use_bias=False))

print((abstracted_model_weights[0]))
ab_model.set_weights(abstracted_model_weights)
ab_model.summary()

[[ 0.14215901 -0.13681966  0.14215901 -0.13681966]
 [ 0.14401719 -0.12869945  0.14401719 -0.12869945]
 [ 0.14765017 -0.13447265  0.14765017 -0.13447265]
 [ 0.14445633 -0.13118415  0.14445633 -0.13118415]
 [ 0.14312282 -0.13909     0.14312282 -0.13909   ]
 [ 0.13731728 -0.1401217   0.13731728 -0.1401217 ]
 [ 0.14017038 -0.14155152  0.14017038 -0.14155152]
 [ 0.13763078 -0.13802716  0.13763078 -0.13802716]
 [ 0.14122036 -0.13192227  0.14122036 -0.13192227]
 [ 0.13262436 -0.13252185  0.13262436 -0.13252185]
 [ 0.13905184 -0.15310161  0.13905184 -0.15310161]
 [ 0.13556994 -0.13169284  0.13556994 -0.13169284]
 [ 0.13712987 -0.14571905  0.13712987 -0.14571905]
 [ 0.15216616 -0.14105581  0.15216616 -0.14105581]
 [ 0.13766539 -0.13996999  0.13766539 -0.13996999]
 [ 0.1457385  -0.14022683  0.1457385  -0.14022683]
 [ 0.14451531 -0.14696236  0.14451531 -0.14696236]
 [ 0.12653039 -0.14498761  0.12653039 -0.14498761]
 [ 0.13102509 -0.1440357   0.13102509 -0.1440357 ]
 [ 0.12899637 -0.14343531  0.12

Need to :change softmax to relu, change no. of final node to 1

#CEGAR

note: _ might need to be reconsidered when refining. If issue occurs, change _ to 0 in abstraction part. Alter mymax etc.

In [ ]:
#othermethod in copy of megenn99


Aux

In [ ]:
#removed

In [20]:

import z3
global z3_vars, z3_assertions
z3_vars = list()

def multiply_matrix(A, B):
    """
    Assuming valid matrix dimensions
    """
    result = [[0 for j in range(len(B[0]))] for i in range(len(A))]
    for i in range(len(A)): 
        for j in range(len(B[0])): 
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return result

def add_matrix(A, B):
    result = [[0 for j in range(len(A[0]))] for i in range(len(A))]
    for i in range(len(A)): 
        for j in range(len(A[0])): 
            result[i][j] = A[i][j] + B[i][j]
            
    return result

def apply_relu(out):
    """
    Returns a new z3 list of variables on which relu is applied with name
    """
#     result = [z3.Real('out_'+ str(i)) for i in range(len(out))]
    curr = [[z3.If(i[0]>0, i[0], 0)] for i in out]
#     for i in range(len(result)):
#         sol.add(result[i] == curr[i][0])
    return curr

def feedforward_z3(weights, x_z3):
    """
    Return the output of the network for the input 'x' which is a list of z3 variables
    """
    layered_output = list()
    out = x_z3
    count = 1
    for w in weights:
        print("Working on layer:", count, "...")
        out0 = multiply_matrix(w, out)
        out = apply_relu(out0)
        layered_output.append(out)
        count += 1
    return out

                
def get_solver(weights): # for relu activation on each layer
    ## input shape can be obtained by the first weights matrix and output shape by the last weights matrix
    weights = np.array(weights)
    weights1 = []
    for i in weights:
      tran = np.transpose(i)
      weights1.append(tran.tolist())

    weights = weights1
    sol = z3.Optimize()
    inp_size = len(weights[0][0])  ### number of columns in the first row of the first matrix
    out_size = len(weights[-1]) ### number of rows in the last weight matrix
    inp_z3 = [[z3.Real('x_' + str(i))] for i in range(inp_size)]
    out_z3 = [[z3.Real('y_' + str(i))] for i in range(out_size)]
    res = feedforward_z3(weights, inp_z3)

    for i in range(len(out_z3)):
        sol.add(res[i][0] == out_z3[i][0])
    return sol, inp_z3, out_z3
    ## let us calculate the output layer by layer using the weights and biases with relu activation

def get_solver2(weights,solv): # for relu activation on each layer
    ## input shape can be obtained by the first weights matrix and output shape by the last weights matrix
    weights = np.array(weights)
    weights1 = []
    for i in weights:
      tran = np.transpose(i)
      weights1.append(tran.tolist())

    weights = weights1
    inp_size = len(weights[0][0])  ### number of columns in the first row of the first matrix
    out_size = len(weights[-1]) ### number of rows in the last weight matrix
    inp_z31 = [[z3.Real('xp_' + str(i))] for i in range(inp_size)]
    out_z31 = [[z3.Real('yp_' + str(i))] for i in range(out_size)]
    res = feedforward_z3(weights, inp_z31)

    for i in range(len(out_z31)):
        solv.add(res[i][0] == out_z31[i][0])
    return solv, inp_z31, out_z31
    ## let us calculate the output layer by layer using the weights and biases with relu activation

some testing

In [ ]:
#ts = [[[1,-1],[0,1]],[[1],[123]]]
#get_solver(ts)


check-sat:

In [96]:
def verify_square_ex(weights,ex):
  print('Checking if spurious')
  weights = np.array(weights)
  weights1 = []
  for i in weights:
    tran = np.transpose(i)
    weights1.append(tran.tolist())
  weights = weights1
  sol = z3.Optimize()
  inp_size = len(weights[0][0])  ### number of columns in the first row of the first matrix

  out_size = len(weights[-1]) ### number of rows in the last weight matrix
  inp_z3 = [[z3.Real('x_' + str(i))] for i in range(inp_size)]
  out_z3 = [[z3.Real('y_' + str(i))] for i in range(out_size)]
  res = feedforward_z3(weights, inp_z3)

#  print('res:',len(res),'\n',res)
#  print('\nloutptx3:',len(output_z3))
  for i in range(len(out_z3)):
    sol.add(res[i][0]==out_z3[i][0])
  print(len(inp_z3))
  print(len(ex))
  print(ex)
  for j in range(len(ex)):
    sol.add(inp_z3[j][0]==ex[j][0])
  
  sol.add(out_z3[0][0]>0)
  return sol.check()

In [97]:
def verify_square(weights):
  sol,input_z3,output_z3 = get_solver(weights)
    #res[i][j] for mid layers, x[i][0]
  squares=[]
  for x in range(15):
    for y in range(15):
      t = min(14-x,14-y)
      for k in range(2,t):
        l_bd=[]            #x,y,k can represent a square whose diagonal is b/w (x,y) & (x+k,y+k), k>=2
        for z in range(k):
          l_bd.append((x,y+z))
          l_bd.append((x+z,y+k))
          l_bd.append((x+k,y+k-z))
          l_bd.append((x+k-z,y))
        #the (x,y)th co-ordinate when in list form is 15*x+y
        imlist1=[]
        imlist2=[]
        for i in range(256):
          imlist1.append(0)
          imlist2.append(255)
      
        for p in l_bd:
          v=(15*p[0])+p[1]
          imlist1[v]=255
          imlist2[v]=0
        #we want [prop(0,i)] to be like one of these lists
        ps1=[]
        ps2=[]
        for i in range(225):
          pv1=imlist1[i]
          pv2=imlist2[i]
          ps1.append(input_z3[i][0]==pv1)
          ps2.append(input_z3[i][0]==pv2)
        squares.append(And(ps1))
        squares.append(And(ps2))
  sol.add(Or(squares))


  #post-condition------------------------------------------
  sol.add(output_z3[0][0]>0) #if >0, means not square
  #---------
  t = sol.check()
  #print(type(str(t)),str(t))
  if str(t)=='unsat':
    return t,input_z3
  else:
    return sol.model(),input_z3

In [48]:
#assumption.. at least 1 hidden layer
split_mod = Sequential()
inlen = len(split_wm[0])
l1 = len(split_wm[1])
#print(inlen,l1)
split_mod.add(Dense(l1, input_dim=inlen, activation='relu',use_bias=False))
for i in range(2,len(split_wm)):
#  print(len(abstracted_model_weights[i]))
  split_mod.add(Dense(len(split_wm[i]),activation='relu',use_bias=False))
lf= len(split_wm[-1][0])
split_mod.add(Dense(lf,activation='relu',use_bias=False))

arified=[]
for i in split_wm:
  arified.append(np.array(i))
#print(arified)
#print(len(arified))
#abstracted_model_weights = arified
print((arified[0]))
split_mod.set_weights(arified)
split_mod.summary()
print('last 2 layers of split_mod:', split_mod.get_weights()[-2:])

[[ 0.03612703  0.03612703  0.03612703 ...  0.04504914  0.04504914
   0.04504914]
 [-0.00914417 -0.00914417 -0.00914417 ... -0.08944418 -0.08944418
  -0.08944418]
 [ 0.01313223  0.01313223  0.01313223 ...  0.07623555  0.07623555
   0.07623555]
 ...
 [ 0.07438732  0.07438732  0.07438732 ... -0.09165407 -0.09165407
  -0.09165407]
 [-0.07907185 -0.07907185 -0.07907185 ... -0.0928677  -0.0928677
  -0.0928677 ]
 [ 0.04501571  0.04501571  0.04501571 ...  0.05363517  0.05363517
   0.05363517]]
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_72 (Dense)             (None, 400)               90000     
_________________________________________________________________
dense_73 (Dense)             (None, 158)               63200     
_________________________________________________________________
dense_74 (Dense)             (None, 36)                5688      
________________________

In [31]:
lwa,lwain=verify_square(split_wm)
lwa#no abstraction here;timeout?

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Working on layer: 1 ...
Working on layer: 2 ...
Working on layer: 3 ...
Working on layer: 4 ...
Working on layer: 5 ...
Working on layer: 6 ...


[]

In [ ]:
#time out.. then don't run this cell
eg1=[]
egt=[]
for i in range(225):
  val1 = lwa[lwain[i][0]].as_long()
  eg1.append([val1])
  egt.append(val1)
print(eg1)
print(split_mod.predict([egt]))
base1 = (np.zeros((15,15)))
for i in range(15):
  for j in range(15):
    base1[i][j] = eg1[(15*i)+j][0]

im1 = Image.fromarray(np.uint8(base1) , 'L')
display(im1)

CEGAR

In [98]:
def cegar(ognetwork,triple):#without splitting 
  abnetwork,ab_part = abstraction(ognetwork,triple) #initial over-approx
 
  
  isize=len(ognetwork[0])


  nlayer=len(ognetwork)
  ognum=0
  for i in range(nlayer):
    ognum+=len(ognetwork[i])

  #ognum gives the number of neurons in the original network (except for the last layer, but that's the same for both)
  t= True
  while t==True:
  #while partition!=ab_part (+1?)
    ablayer=len(abnetwork)
    abnum=0
    for i in range(ablayer):
      abnum+=len(abnetwork[i])
    if ognum<=abnum:
      t=False    

    v,input_z3 = verify_square(abnetwork)
    if str(v) == 'unsat':#nom/...
      return v,[]

    else:
      #find cex  v.. contains this.
      cex=[]
      for i in range(225):
        val = v[input_z3[i][0]].as_long()
        cex.append([val])
#      print('doneloop:\n',cex ,'\n',len(cex))
      print(len(ognetwork[0]))
      check_og = verify_square_ex(ognetwork,cex)
      print('...!:',check_og)
      if str(check_og) == 'sat':
        return check_og,cex

      else:


        abnetwork,ab_part = refine(ognetwork,triple,abnetwork,ab_part,cex)
      print('refined')
      #if number of nodes is the same b/w the two then stop.
  print('exitloop')



result, cex1 = cegar(split_wm,split_infox)

#running.. refine seems to happen quickly enough.. checking not so much
#need to ensure spurious as well

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Working on layer: 1 ...
Working on layer: 2 ...
Working on layer: 3 ...
Working on layer: 4 ...
Working on layer: 5 ...
Working on layer: 6 ...
225
Checking if spurious
Working on layer: 1 ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


Working on layer: 2 ...
Working on layer: 3 ...
Working on layer: 4 ...
Working on layer: 5 ...
Working on layer: 6 ...
225
225
[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [255], [255], [255], [255], [255], [255], [0], [0], [0], [0], [0], [0], [0], [0], [0], [255], [0], [0], [0], [0], [255], [0], [0], [0], [0], [0], [0], [0], [0], [0], [255], [0], [0], [0], [0], [255], [0], [0], [0], [0], [0], [0], [0], [0], [0], [255], [0], [0], [0], [0], [255], [0], [0], [0], [0], [0], [0], [0], [0], [0], [255], [0], [0], [0], [0], [255], [0], [0], [0], [0], [0], [0], [0], [0], [0], [255], [255], [255], [255], [255], [255], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [

In [99]:

base1 = (np.zeros((15,15)))
for i in range(15):
  for j in range(15):
    base1[i][j] = cex1[(15*i)+j][0]

im1 = Image.fromarray(np.uint8(base1) , 'L')
display(im1)

cex2=[]
for i in cex1:
  cex2.append(i[0])
print(split_mod.predict([cex2]))



print(model.predict([cex2]))

[[0.20521832]]
[[1.1573144e-03 9.5623873e-06 5.5048215e-01 4.4835091e-01]]


-----------------------------------Rotation Attempt---------------------------------------------

In [212]:
#len(model.get_weights()[0])
weight_mod = model.get_weights()
is0 = isclasskmaker(4,2)
for i in is0:
  weight_mod.append(i)
print('model with 1 output:',weight_mod)
split_wm,split_infox = splitnetwork(weight_mod)

model with 1 output: [array([[ 3.6127035e-02,  5.6199886e-02,  6.6610478e-02, ...,
         4.4020712e-02,  9.6576318e-02,  4.5049138e-02],
       [-9.1441702e-03,  6.9404088e-02, -5.7055060e-02, ...,
        -3.5717983e-02,  7.3694386e-02, -8.9444183e-02],
       [ 1.3132227e-02, -4.6205945e-02,  5.9141043e-05, ...,
        -1.1069778e-01, -5.8497082e-02,  7.6235548e-02],
       ...,
       [ 7.4387319e-02, -2.3392579e-02,  1.1829632e-01, ...,
         7.7394904e-03,  1.2969336e-01, -9.1654070e-02],
       [-7.9071850e-02,  4.4572237e-03, -1.6070472e-02, ...,
        -7.7395856e-02,  1.2861602e-01, -9.2867702e-02],
       [ 4.5015711e-02,  1.0438054e-02,  7.8057490e-02, ...,
        -2.5746576e-02, -3.4494568e-02,  5.3635169e-02]], dtype=float32), array([[ 0.19373448, -0.18705568, -0.10012381, ..., -0.17807184,
         0.1522985 , -0.00381974],
       [-0.17776063,  0.17617904, -0.1218728 , ..., -0.16635543,
         0.13369158, -0.02622381],
       [ 0.16712669, -0.09214905, -0.1404

In [213]:
#assumption.. at least 1 hidden layer
split_mod = Sequential()
inlen = len(split_wm[0])
l1 = len(split_wm[1])
#print(inlen,l1)
split_mod.add(Dense(l1, input_dim=inlen, activation='relu',use_bias=False))
for i in range(2,len(split_wm)):
#  print(len(abstracted_model_weights[i]))
  split_mod.add(Dense(len(split_wm[i]),activation='relu',use_bias=False))
lf= len(split_wm[-1][0])
split_mod.add(Dense(lf,activation='relu',use_bias=False))

arified=[]
for i in split_wm:
  arified.append(np.array(i))
print(arified)
#print(len(arified))
#abstracted_model_weights = arified
print((arified[0]))
split_mod.set_weights(arified)
split_mod.summary()

[array([[ 0.03612703,  0.03612703,  0.03612703, ...,  0.04504914,
         0.04504914,  0.04504914],
       [-0.00914417, -0.00914417, -0.00914417, ..., -0.08944418,
        -0.08944418, -0.08944418],
       [ 0.01313223,  0.01313223,  0.01313223, ...,  0.07623555,
         0.07623555,  0.07623555],
       ...,
       [ 0.07438732,  0.07438732,  0.07438732, ..., -0.09165407,
        -0.09165407, -0.09165407],
       [-0.07907185, -0.07907185, -0.07907185, ..., -0.0928677 ,
        -0.0928677 , -0.0928677 ],
       [ 0.04501571,  0.04501571,  0.04501571, ...,  0.05363517,
         0.05363517,  0.05363517]], dtype=float32), array([[ 0.19373448,  0.        ,  0.19373448, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.19373448,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ..., -0.00381974,
         0.        , -0.00381974],
       ...,
       [ 0.        ,  0.06540401,  0.        , ...,  0.   

In [214]:
def verify_rot(weights):
#  s= z3.Solver()
  sol,input_z3,output_z3 = get_solver(weights)
  sol1,input_z31,output_z31 = get_solver2(weights,sol)
  #pre-condition-------------------------------------------

  #90 rotation given by (15x+y)->(210-15y+x)
  for xu in range(15):
    for yu in range(15):
      i=(15*xu)+yu
      j=210-(15*yu)+xu
      sol1.add(input_z3[i][0]==input_z31[j][0])


  for i in range(225):
    sol1.add(Or(input_z3[i][0]==0,input_z3[i][0]==255))
    #we're taking two images, with the second being a 90 degree clockwise rotation of the previous

  #post-condition------------------------------------------
  ne = []
  for i in range(225):
    ne.append(input_z3[i][0]==0)
  sol1.add(Not(And(ne)))

  p1 = And(output_z3[0][0]>0,output_z31[0][0]>0)

  #sol1.add(sol,sol1)
  sol1.add(p1)


  #---------
  t = sol1.check()
  if str(t)=='unsat':
    return t,input_z3
  else:
    return sol1.model(),input_z3

In [215]:
def verify_rot_ex(weights,ex):
  print('Checking if spurious')
  weights = np.array(weights)
  weights1 = []
  for i in weights:
    tran = np.transpose(i)
    weights1.append(tran.tolist())
  weights = weights1
  sol = z3.Optimize()
  inp_size = len(weights[0][0])  ### number of columns in the first row of the first matrix

  out_size = len(weights[-1]) ### number of rows in the last weight matrix
  inp_z3 = [[z3.Real('x_' + str(i))] for i in range(inp_size)]
  out_z3 = [[z3.Real('y_' + str(i))] for i in range(out_size)]
  res = feedforward_z3(weights, inp_z3)

  inp_z31 = [[z3.Real('xp_' + str(i))] for i in range(inp_size)]
  out_z31 = [[z3.Real('yp_' + str(i))] for i in range(out_size)]
  res1 = feedforward_z3(weights, inp_z3)
#  print('res:',len(res),'\n',res)
#  print('\nloutptx3:',len(output_z3))
  for i in range(len(out_z3)):
    sol.add(res[i][0]==out_z3[i][0])
    sol.add(res1[i][0]==out_z31[i][0])
  print(len(inp_z3))
  print(len(ex))
  print(ex)
  for xu in range(15):
    for yu in range(15):
      i=(15*xu)+yu
      j=210-(15*yu)+xu
      sol.add(inp_z3[i][0]==ex[i][0])
      sol.add(inp_z31[j][0]==ex[i][0])
  
  sol.add(out_z3[0][0]>0,out_z31[0][0]>0)
  return sol.check()

In [216]:
def cegar_rot(ognetwork,triple):#without splitting 
  abnetwork,ab_part = abstraction(ognetwork,triple) #initial over-approx
 
  
  isize=len(ognetwork[0])


  nlayer=len(ognetwork)
  ognum=0
  for i in range(nlayer):
    ognum+=len(ognetwork[i])

  #ognum gives the number of neurons in the original network (except for the last layer, but that's the same for both)
  t= True
  while t==True:
  #while partition!=ab_part (+1?)
    ablayer=len(abnetwork)
    abnum=0
    for i in range(ablayer):
      abnum+=len(abnetwork[i])
    if ognum<=abnum:
      t=False    

    v,input_z3 = verify_rot(abnetwork)
    if str(v) == 'unsat':#nom/...
      print('unsat case')
      return v,[]

    else:
      #find cex  v.. contains this.
      cex=[]
#      print('v:',v)
      for i in range(225):
        val = v[input_z3[i][0]].as_long()
        cex.append([val])
#      print('doneloop:\n',cex ,'\n',len(cex))
      print(len(ognetwork[0]))
      check_og = verify_square_ex(ognetwork,cex)
      print('...!:',check_og)
      if str(check_og) == 'sat':
        return check_og,cex

      else:


        abnetwork,ab_part = refine(ognetwork,triple,abnetwork,ab_part,cex)
      print('refined')
      #if number of nodes is the same b/w the two then stop.
  print('exitloop')



result, cex1 = cegar_rot(split_wm,split_infox)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Working on layer: 1 ...
Working on layer: 2 ...
Working on layer: 3 ...
Working on layer: 4 ...
Working on layer: 5 ...
Working on layer: 6 ...
Working on layer: 1 ...
Working on layer: 2 ...
Working on layer: 3 ...
Working on layer: 4 ...
Working on layer: 5 ...
Working on layer: 6 ...
225
Checking if spurious
Working on layer: 1 ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


Working on layer: 2 ...
Working on layer: 3 ...
Working on layer: 4 ...
Working on layer: 5 ...
Working on layer: 6 ...
225
225
[[255], [255], [0], [255], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [255], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [255], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [255], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [255], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [255], [0], [0], [255], [255], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [

In [209]:

base1 = (np.zeros((15,15)))
#cex1=[[255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [0], [0], [0], [0], [0], [0], [255], [255], [255], [255], [255], [255], [255], [255], [255], [0], [255], [255], [255], [255], [0], [255], [255], [255], [255], [255], [255], [255], [255], [255], [0], [255], [255], [255], [255], [0], [255], [255], [255], [255], [255], [255], [255], [255], [255], [0], [255], [255], [255], [255], [0], [255], [255], [255], [255], [255], [255], [255], [255], [255], [0], [255], [255], [255], [255], [0], [255], [255], [255], [255], [255], [255], [255], [255], [255], [0], [0], [0], [0], [0], [0], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255], [255]]
for i in range(15):
  for j in range(15):
    base1[i][j] = cex1[(15*i)+j][0]

im1 = Image.fromarray(np.uint8(base1) , 'L')
display(im1)

cex2=[]
for i in cex1:
  cex2.append(i[0])
print(split_mod.predict([cex2]))



print(model.predict([cex2]))

[[21.164223]]
[[1.7746563e-19 5.4168185e-34 8.0692321e-01 1.9307680e-01]]


In [210]:
base1 = (np.zeros((15,15)))
cexd=[]
for i in cex1:
  cexd.append(i)
for i in range(15):
  for j in range(15):
    cexd[(15*i)+j] = cex1[210-(15*j)+i]
for i in range(15):
  for j in range(15):
    base1[i][j] = cexd[15*i+j][0]
im1 = Image.fromarray(np.uint8(base1) , 'L')
display(im1)

cex2=[]
for i in cexd:
  cex2.append(i[0])
print(split_mod.predict([cex2]))



print(model.predict([cex2]))#

[[7.4088774]]
[[1.8314260e-07 1.5186447e-12 6.2261039e-01 3.7738943e-01]]
